## Etapa 1: Instalação da bibliotecas


In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 87kB/s 
     |████████████████████████████████| 419kB 39.4MB/s 
     |████████████████████████████████| 3.0MB 25.3MB/s 


In [0]:
!pip install pandas-datareader

## Etapa 2: Importação das bibliotecas

In [0]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [0]:
tf.__version__

'2.0.0-alpha0'

## Etapa 3: Construção da IA para negociação de ações

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen = 2000)
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.model_builder()
    
  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
    model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
    return model
  
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay
      

## Etapa 4: Pré-processamento da base de dados

### Definição de funções auxiliares

#### Sigmoid

In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [0]:
sigmoid(0.5)

0.6224593312018546

#### Formatação de preços

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [0]:
stocks_price_format(100)

'$ 100.000000'

#### Carregador da base de dados

In [0]:
dataset = data_reader.DataReader("AAPL", data_source = "yahoo")

In [0]:
dataset.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,30.642857,30.340000,30.490000,30.572857,123432400.0,26.681330
2010-01-05,30.798571,30.464285,30.657143,30.625713,150476200.0,26.727465
2010-01-06,30.747143,30.107143,30.625713,30.138571,138040000.0,26.302330
2010-01-07,30.285715,29.864286,30.250000,30.082857,119282800.0,26.253704
2010-01-08,30.285715,29.865715,30.042856,30.282858,111902700.0,26.428249


In [0]:
str(dataset.index[0]).split()[0]

'2010-01-04'

In [0]:
dataset.index[-1]

Timestamp('2019-09-17 00:00:00')

In [0]:
dataset['Close']

Date
2010-01-04     30.572857
2010-01-05     30.625713
2010-01-06     30.138571
2010-01-07     30.082857
2010-01-08     30.282858
2010-01-11     30.015715
2010-01-12     29.674286
2010-01-13     30.092857
2010-01-14     29.918571
2010-01-15     29.418571
2010-01-19     30.719999
2010-01-20     30.247143
2010-01-21     29.724285
2010-01-22     28.250000
2010-01-25     29.010000
2010-01-26     29.420000
2010-01-27     29.697144
2010-01-28     28.469999
2010-01-29     27.437143
2010-02-01     27.818571
2010-02-02     27.980000
2010-02-03     28.461428
2010-02-04     27.435715
2010-02-05     27.922857
2010-02-08     27.731428
2010-02-09     28.027143
2010-02-10     27.874287
2010-02-11     28.381428
2010-02-12     28.625713
2010-02-16     29.057142
                 ...    
2019-08-06    197.000000
2019-08-07    199.039993
2019-08-08    203.429993
2019-08-09    200.990005
2019-08-12    200.479996
2019-08-13    208.970001
2019-08-14    202.750000
2019-08-15    201.740005
2019-08-16    206.50

In [0]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

### Criador de estados

In [0]:
0 - 5 + 1

-4

In [0]:
20 - 5 + 1

16

In [0]:
dataset[16:21]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-27,30.082857,28.504286,29.549999,29.697144,430642100.0,25.917091
2010-01-28,29.357143,28.385714,29.275715,28.469999,293375600.0,24.846146
2010-01-29,28.885714,27.178572,28.725714,27.437143,311488100.0,23.944765
2010-02-01,28.000000,27.328571,27.481428,27.818571,187469100.0,24.277634
2010-02-02,28.045713,27.625713,27.987143,27.980000,174585600.0,24.418516


In [0]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
    
  return np.array([state]), windowed_data

### Carregando a base de dados

In [0]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [0]:
s, w = state_creator(data, 0, 5)

In [0]:
s

array([[0.5, 0.5, 0.5, 0.5]])

In [0]:
w

[30.572856903076172,
 30.572856903076172,
 30.572856903076172,
 30.572856903076172,
 30.572856903076172]

## Etapa 5: Treinando a IA

### Configuração dos hyper parâmetros

In [0]:
window_size = 10
episodes = 1000
batch_size = 32
data_samples = len(data) - 1

In [0]:
data_samples

2442

### Definição do modelo

In [0]:
trader = AI_Trader(window_size)

In [0]:
trader.model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_8 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_9 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Loop de treinamento

In [0]:
for episode in range(1, episodes + 1):
  print("Episode: {}/{}".format(episode, episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0
    
    if action == 1: # Comprando uma ação
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: # Vendendo uma ação  
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("Total profit: {}".format(total_profit))
      print("########################")
      
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
     
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/2442 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 30.138571
AI Trader bought:  $ 30.015715
AI Trader bought:  $ 30.092857
AI Trader bought:  $ 29.918571
AI Trader bought:  $ 29.418571
AI Trader sold:  $ 30.719999  Profit: $ 0.581429
AI Trader bought:  $ 29.724285
AI Trader bought:  $ 29.010000
AI Trader sold:  $ 29.697144  Profit: - $ 0.318571
AI Trader sold:  $ 27.437143  Profit: - $ 2.655714
AI Trader bought:  $ 27.818571
AI Trader sold:  $ 28.461428  Profit: - $ 1.457144
AI Trader sold:  $ 27.435715  Profit: - $ 1.982857
AI Trader sold:  $ 27.731428  Profit: - $ 1.992857
AI Trader bought:  $ 28.625713
AI Trader sold:  $ 29.057142  Profit: $ 0.047142
AI Trader bought:  $ 28.990000


  1%|▏         | 34/2442 [00:01<04:22,  9.16it/s]

AI Trader bought:  $ 28.151428


  1%|▏         | 36/2442 [00:02<07:19,  5.48it/s]

...

  1%|▏         | 35/2442 [00:09<10:48,  3.71it/s]

AI Trader sold:  $ 28.665714  Profit: $ 0.514286


  3%|▎         | 78/2442 [00:20<10:09,  3.88it/s]

AI Trader bought:  $ 37.434284


KeyboardInterrupt: ignored